In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from tempfile import TemporaryDirectory
import imageio
from functools import partial
import gpytorch

In [ ]:
# Import the campaign objects
from sva.campaign import Campaign, CampaignData, FixedSVAPolicy

# Import the various experiments we need for the notebook
from sva.experiments import Sine2Phase

# Import the model
from sva.models import EasySingleTaskGP

# Utils
from sva.mpl_utils import set_mpl_defaults

In [ ]:
set_mpl_defaults()

# A simple 2d example

Two-phase experiment with a sinusoidal phase boundary.

In [ ]:
experiment = Sine2Phase()
extent = experiment.get_domain_mpl_extent()

In [ ]:
# Get the ground truth results
x = experiment.get_dense_coordinates(ppd=100)
y = experiment.get_phase(x)

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(2, 2), sharex=True, sharey=True)

ax = axs
ax.imshow(
    y.reshape(100, 100).T,
    extent=extent,
    interpolation="nearest",
    origin="lower",
    cmap="RdBu",
)

plt.show()

# Running the experiment

In [ ]:
N_start = 3
N_max = 150
SEED = 133

In [ ]:
data = CampaignData()
data.prime(experiment, "LatinHypercube", seed=SEED, n=N_start)
covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel())
model_factory = partial(EasySingleTaskGP.from_default, covar_module=covar_module)
policy = FixedSVAPolicy(
    n_max=N_max, acquisition_function="UCB-50", model_factory=model_factory, save_model=True,
    optimize_kwargs={"q": 1, "num_restarts": 20, "raw_samples": 100}
)
campaign = Campaign(data=data, experiment=experiment, policy=policy, seed=SEED)

In [ ]:
campaign.run()

In [ ]:
grid = np.linspace(0, 1, 100)
b = 0.5 + 0.25 * np.sin(2.0 * np.pi * grid)

In [ ]:
gp = data.metadata[-1]["model"]
mu, var = gp.predict(x)
mu = mu.reshape(100, 100).T
mu -= mu.min()
mu /= mu.max()
var = var.reshape(100, 100).T
var -= var.min()
var /= var.max()

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(2, 2), sharex=True, sharey=True)

ax = axs

X = campaign.data.X
im = ax.imshow(mu, extent=extent, interpolation="nearest", origin="lower")
ax.scatter(X[:, 0], X[:, 1], color="red", s=0.5, zorder=3)
ax.plot(grid, b, linestyle="--", linewidth=0.5, color="black")

ax.set_xticks([0, 1])
ax.set_yticks([0, 1])

ax.set_xlabel("$x_1$")
ax.set_ylabel("$x_2$")

plt.show()

## Let's make a GIF of the mean

In [ ]:
X = data.X

with TemporaryDirectory() as f:
    filenames = []

    for ii, datum in tqdm(enumerate(campaign.data.metadata)):
        gp = datum.get("model")
        if not gp:
            continue
        mu, var = gp.predict(x)
        mu = mu.reshape(100, 100).T
        mu -= mu.min()
        mu /= mu.max()

        fig, axs = plt.subplots(1, 1, figsize=(3, 3), sharex=True, sharey=True)

        ax = axs

        n = ii
        im = ax.imshow(
            mu, extent=extent, interpolation="nearest", origin="lower"
        )
        ax.scatter(X[:n, 0], X[:n, 1], color="red", s=0.5, zorder=3)
        ax.plot(grid, b, linestyle="--", linewidth=0.5, color="black")

        ax.set_xticks([0, 1])
        ax.set_yticks([0, 1])

        ax.set_xlabel("$x_1$")
        ax.set_ylabel("$x_2$")

        ax.set_title(f"$N={n}$")

        cbar = plt.colorbar(im, ax=ax)
        cbar.set_ticks([0, 1])
        cbar.set_label(r"$\mathcal{G}(\mathbf{x}) \approx U_i(D_N)$")

        filename = f"{f}/{ii}.png"
        filenames.append(filename)

        plt.savefig(filename)
        plt.close()

    frames = []
    for filename in filenames:
        frames.append(imageio.imread(filename))

    exportname = "sine2phase.gif"
    kwargs = {"duration": 1.0}
    imageio.mimsave(exportname, frames, "GIF", **kwargs)